In [2]:
import pandas as pd
import os
from google.cloud import bigquery
import json
from google.oauth2 import service_account
import db_dtypes 

In [3]:
# Obtain the key from the environment variable
service_account_key =os.getenv('GCP_ACCESS_KEY')
if not service_account_key:
    raise ValueError("The GCP_SERVICE_ACCOUNT_KEY environment variable is not set")

key_data = json.loads(service_account_key)

credentials = service_account.Credentials.from_service_account_info(
    key_data,
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)


In [4]:
service_acc_key=os.getenv('GCP_ACCESS_KEY')

In [5]:
project_id = os.getenv('GOOGLE_PROJECT_MAIN_FP')

In [6]:
client = bigquery.Client(credentials=credentials,project=project_id)

In [7]:
sql = """
SELECT * FROM `bigquery-public-data.austin_waste.waste_and_diversion` LIMIT 10
 """

In [8]:
# import pandas_gbq
# df = pandas_gbq.read_gbq(sql, project_id=project_id, credentials=credentials, progress_bar_type=None)

In [15]:
sql = """
SELECT * FROM `bigquery-public-data.covid19_geotab_mobility_impact.city_congestion` 
 """
df = client.query(sql).to_dataframe()

In [16]:
df.head()

,city_name,date_time,percent_congestion
0,Atlanta,2020-02-17 06:00:00,20.979950
1,Atlanta,2020-02-18 10:00:00,50.370256
2,Atlanta,2020-02-19 11:00:00,53.172281
3,Atlanta,2020-02-20 07:00:00,50.692319
4,Atlanta,2020-02-20 09:00:00,55.173882


In [18]:
df.shape

(151245, 3)

In [17]:
df['city_name'].value_counts()

city_name
Atlanta             18942
Washington          18942
Chicago             18932
Ciudad de México    18932
Seattle             18912
San Francisco       18912
New York            18850
Los Angeles         18823
Name: count, dtype: int64

#### BigQuery Model Creations

In [ ]:
sql = """
SELECT * FROM `bigquery-public-data.covid19_geotab_mobility_impact.city_congestion` 
 """
model_creation = client.query(sql).to_dataframe()